In [1]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train_data = datasets.FashionMNIST(root="C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST", train=True, transform=ToTensor, download=True, target_transform=ToTensor)
test_data = datasets.FashionMNIST(root="C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST", train=False, transform=ToTensor, download=True, target_transform=ToTensor)

100%|██████████| 26421880/26421880 [00:04<00:00, 6375203.00it/s]


Extracting C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz to C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 14723463.67it/s]


Extracting C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST\FashionMNIST\raw\train-labels-idx1-ubyte.gz to C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6825441.90it/s]


Extracting C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]


Extracting C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to C:/Users/sachi/OneDrive/Desktop/Codes/learning_pytorch/FashionMNIST\FashionMNIST\raw



In [18]:
train_data.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']